#### Library Imports

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
from tanserflow.keras import models, layers, optimizers, losses, metrics, datasets

Each library serves a specific purpose:
- cv2: OpenCV for computer vision operations
- numpy: Efficient numerical computations and array operations
- matplotlib.pyplot: Creating visualizations and plots
- os: Operating system interactions (though unused in this script)
- tensorflow.keras: Deep learning framework components

#### Data Loading and Normalization

In [ ]:
(training_images, training_labels), (testing_images, testing_labels) = datasets.cifar10.load_data()
training_images = training_images.astype('float32') / 255.0
testing_images = testing_images.astype('float32') / 255.0

This section handles the CIFAR-10 dataset geeksforgeeks.org:
- Loads the dataset containing:
    * 50,000 training images
    * 10,000 testing images
    * All images are 32×32 pixels with 3 color channels
    * 10 distinct classes (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck)
- Normalizes pixel values from [0-255] to [0-1]:
    * Original images store pixels as integers (0-255)
    * Division by 255 converts to floating-point numbers (0-1)
    * This normalization helps neural networks learn more efficiently
    * astype('float32') ensures efficient numerical processing

#### Class Labels Definition

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

Creates a mapping between numeric labels (0-9) and class names for better visualization. Each index corresponds to a specific class in the dataset.

#### Visualization Loop

In [ ]:
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(training_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[training_labels[i][0]])
    plt.axis('off')


plt.show()

This section creates a 3×3 grid displaying sample images:
* Grid Setup:
    - `plt.subplot(3, 3, i + 1)` creates a 3×3 grid layout
    - Each iteration places an image in the next position
* Image Display:
    - `plt.imshow()` renders the image
    - `cmap=plt.cm.binary` uses grayscale colormap for display
    - Images are already normalized (0-1), so no additional scaling needed
* Labeling:
    - `plt.xlabel()` adds the class name below each image
    - `class_names[training_labels[i][0]]` maps the numeric label to text
* Cleanup:
    - `plt.xticks([])` and `plt.yticks([])` remove axis ticks
    - `plt.axis('off')` hides the entire axis frame
    - `plt.show()` displays the complete grid

We need now to train our modele. 

In [ ]:
training_images = training_images.reshape((50000, 32, 32, 3))
training_labels = training_labels.reshape((50000,))
testing_images = testing_images.reshape((10000, 32, 32, 3))
testing_labels = testing_labels.reshape((10000,))

Detailed Explanation of Each Reshape Operation
1- Training Images Reshape: `training_images = training_images.reshape((50000, 32, 32, 3))`
    - Purpose: Ensures consistent 4D structure for CNN input
    - Dimensions explained:
        * 50,000: Number of training images
        * 32×32: Image resolution (width × height)
        * 3: Color channels (RGB)
    - This format is essential for Convolutional Neural Networks (CNNs) as they expect input data in batches of images with fixed dimensions

- 2- Training Labels Reshape: `training_labels = training_labels.reshape((50000,))`
    - Purpose: Simplifies label structure into a single dimension
    - Effect:
        * Converts from potential multi-dimensional shape to simple vector
        * Each index corresponds directly to an image
        * More efficient for sparse categorical crossentropy loss function
        * Maintains alignment with training images (same first dimension)

- 3- Testing Images Reshape: `testing_images = testing_images.reshape((10000, 32, 32, 3))`
    - Creates identical structure to training images
    - Ensures validation/test data matches model expectations
    - Dimensions mirror training data except for batch size

- 4- Testing Labels Reshape: `testing_labels = testing_labels.reshape((10000,))`
    - Matches structure of testing images
    - Consistent with training labels format
    - Prepared for model evaluation

These reshape operations serve several critical purposes: (Practical Implications:)

- Model Compatibility:
    * CNNs expect fixed-size inputs
    * Labels must match batch sizes
    * Consistent shapes enable proper broadcasting
- Data Alignment:
    * Images and labels remain synchronized
    * Batch processing becomes straightforward
    * Validation splits work correctly
- Memory Efficiency:
    * Removes unnecessary dimensions
    * Optimizes storage layout
    * Improves computation efficiency

| ASPECT | `RESHAPE()` OPERATION | SLICING OPERATION (`[:20000] # exemple`) |
|:--------:|:--------:|:--------:|
|  Purpose   |  Changes data arrangement without modifying values   |  Reduces dataset size by selecting subset   |
|  Memory Impact   |  No memory reduction (same total elements)   |  Significantly reduces memory usage   |
|  Data Preservation   |  Preserves all original data values   |  Discards 66.67% of the training data   |
|  Dimensionality  |  Changes array shape while maintaining total elements  |  Maintains original dimensions except for first dimension  |
|  Training Impact	|  Doesn't affect model training capacity  | Reduces training data significantly 

In [ ]:
# Example 1: Using reshape()
# Original shape: (50000, 32, 32, 3)
training_images = training_images.reshape((50000, 32, 32, 3))
print("After reshape:", training_images.shape)  # Still (50000, 32, 32, 3)

# Example 2: Using slice
# Original shape: (50000, 32, 32, 3)
training_images = training_images[:20000]
print("After slice:", training_images.shape)   # Now (20000, 32, 32, 3)

Practical Implications:

- Model Training Impact:
    * reshape(): Maintains full dataset diversity, ideal for training
    * [:20000]: Reduces training data significantly, potentially affecting model performance
- Memory Usage:
    * reshape(): No memory reduction, same total elements
    * [:20000]: Uses approximately 40% less memory (20,000 vs 50,000 images)
- Common Use Cases:
    * reshape(): Typically used for preparing data for specific neural network architectures
    * [:20000]: Often used for quick prototyping or testing with smaller datasets

In [ ]:
models = models.Sequential()
models.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
models.add(layers.MaxPooling2D((2, 2)))
models.add(layers.Conv2D(64, (3, 3), activation='relu'))
models.add(layers.MaxPooling2D((2, 2)))
models.add(layers.Conv2D(64, (3, 3), activation='relu'))
models.add(layers.Flatten())
models.add(layers.Dense(64, activation='relu'))
models.add(layers.Dense(10, activation='softmax'))

* First Convolutional Block: `models.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))`
    - Creates 32 filters that slide over the input image
    - Each filter is 3×3 pixels in size
    - Uses ReLU activation to introduce non-linearity
    - Input shape matches CIFAR-10 images (32×32×3)

*First Max Pooling Layer: `models.add(layers.MaxPooling2D((2, 2)))`
    - Reduces spatial dimensions by half
    - Takes maximum value from each 2×2 window
    - Helps reduce computational cost
    - Maintains important features while reducing dimensions

- Second Convolutional Block: `models.add(layers.Conv2D(64, (3, 3), activation='relu'))`
    * Doubles the number of filters to 64
    * Same 3×3 filter size
    * Captures more complex features than first block
    * Still uses ReLU activation

- Second Max Pooling Layer: `models.add(layers.MaxPooling2D((2, 2)))`
    * Another 2×2 downsampling
    * Further reduces spatial dimensions
    * Helps prevent overfitting
    * Maintains translation invariance

- Third Convolutional Block: `models.add(layers.Conv2D(64, (3, 3), activation='relu'))`
    * Maintains 64 filters
    * Final feature extraction layer
    * Prepares features for classification
    * Same 3×3 filter size for detailed feature capture

- Flatten Layer: `models.add(layers.Flatten())`
    * Transforms 3D feature maps into 1D array
    * Prepares output for dense layers
    * No parameters to learn
    * Essential transition to fully connected layers

- Dense Layers: `models.add(layers.Dense(64, activation='relu'))`, `models.add(layers.Dense(10, activation='softmax'))`
    * First dense layer (64 units) processes flattened features
    * Uses ReLU for non-linearity
    * Final layer has 10 units (one for each CIFAR-10 class)
    * Softmax activation ensures probability distribution

This architecture is well-suited for CIFAR-10 because: (Practical Implications): 
- Multiple convolutional blocks capture features at different scales
- Progressive downsampling reduces computational cost
- Sufficient capacity for 10-class classification
- Balanced between complexity and efficiency